# Question 1

We are going to apply Analysis of Variance (ANOVA) to check if the mean values of proportions are the same for both pairs of groups. 

First of all, we import the required modules.

In [1]:
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.graphics as sgr
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.collections import LineCollection
from statsmodels.stats.outliers_influence import summary_table
from pandas.core import datetools

%matplotlib inline

C:\Users\leo\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


Then, we read the csv file.

In [2]:
workplace = pd.read_csv('Reveal_EEO1_for_2016.csv') #import to a pandas dataframe calles workplace

In [3]:
workplace.head(10) #select first 10 rows

,company,year,race,gender,job_category,count
0,23andMe,2016,Latino,male,Executive/Senior officials & Mgrs,0
1,23andMe,2016,Latino,male,First/Mid officials & Mgrs,1
2,23andMe,2016,Latino,male,Professionals,7
3,23andMe,2016,Latino,male,Technicians,0
4,23andMe,2016,Latino,male,Sales workers,0
5,23andMe,2016,Latino,male,Administrative support,0
6,23andMe,2016,Latino,male,Craft workers,0
7,23andMe,2016,Latino,male,operatives,0
8,23andMe,2016,Latino,male,laborers and helpers,0
9,23andMe,2016,Latino,male,Service workers,0


Then we replace NAs with zeros.

In [4]:
workplace['count'].replace(to_replace='na',value=0,inplace=True)#set zeros
workplace['count']=workplace['count'].astype(int) #replace with zeros

We want to perform a hypothesis testing and check whether two (or more groups) have the same population mean regarding proportions of males-females and white-non white people with senior positions in the workplace, compared to those with non-senior positions, respectivelly. 

The first step towards analysing the data is understanding it. Thus, we select the distinct values of races and job categories that raise interest.

In [5]:
workplace.race.unique() #select distinct values of races

array(['Latino', 'White', 'Black_or_African_American',
       'Native_Hawaiian_or_Pacific_Islander', 'Asian',
       'American_Indian_Alaskan_Native', 'Two_or_more_races',
       'Overall_totals'], dtype=object)

In [6]:
workplace.job_category.unique()#select distinct values of categories

array(['Executive/Senior officials & Mgrs', 'First/Mid officials & Mgrs',
       'Professionals', 'Technicians', 'Sales workers',
       'Administrative support', 'Craft workers', 'operatives',
       'laborers and helpers', 'Service workers', 'Totals',
       'Previous_totals'], dtype=object)

At this point and taking the question requirements into consideration, we decide to split the data according to:


1) Race : 

    'White' for white
    
    'Non-White' for non-white people

2) Job category :

    'Senior' for Senior Positions
        (including: 'Executive/Senior officials & Mgrs', 'First/Mid officials & Mgrs', 'Professionals') 
        
    'Non-Senior' for Non-Senior Positions    
        (including: 'Technicians', 'Sales workers', 'Administrative support', 'Craft workers', 'operatives', 
         'laborers and helpers', 'Service workers', 'Totals', 'Previous_totals')

We add to our dataframe the categorical variables for race and job category.

More specifically, we get two different levels: 0 when race is White and Non-White otherwise.

In [7]:
workplace['c_race'] = np.where(workplace['race'] == 'White', 'White', 'Non-White') 
workplace.head(10)

,company,year,race,gender,job_category,count,c_race
0,23andMe,2016,Latino,male,Executive/Senior officials & Mgrs,0,Non-White
1,23andMe,2016,Latino,male,First/Mid officials & Mgrs,1,Non-White
2,23andMe,2016,Latino,male,Professionals,7,Non-White
3,23andMe,2016,Latino,male,Technicians,0,Non-White
4,23andMe,2016,Latino,male,Sales workers,0,Non-White
5,23andMe,2016,Latino,male,Administrative support,0,Non-White
6,23andMe,2016,Latino,male,Craft workers,0,Non-White
7,23andMe,2016,Latino,male,operatives,0,Non-White
8,23andMe,2016,Latino,male,laborers and helpers,0,Non-White
9,23andMe,2016,Latino,male,Service workers,0,Non-White


We now have to make a distinction according to seniority of positions. 

Since it is not a definition of seniority, we choose Executive/Senior officials & Mgrs, First/Mid officials & Mgrs and Professionals as senior Categories and all the other Non-Senios.

In [8]:
workplace['c_job_category'] = np.where(workplace['job_category'] == 'Executive/Senior officials & Mgrs' , 'Senior' , 'Non-Senior')
workplace['c_job_category'] = np.where(workplace['job_category'] == 'First/Mid officials & Mgrs' , 'Senior' , workplace['c_job_category'])
workplace['c_job_category'] = np.where(workplace['job_category'] == 'Professionals' , 'Senior' , workplace['c_job_category'])

Now, we split wordplace into two different dataframes according to seniority level.
In each dataframe we will calculate the proportion of males and females for each company. 
Then we will join two dataframes on company column and will perform ANOVA Analysis on the final dataframe that will contain proportions for males in Senior and Non-Senior positions.

In [9]:
d1 = workplace[workplace['c_job_category'] == 'Senior'].reset_index() #d1 contains seniors positions.reset index to perform a group by statement
d1.head(10)

,index,company,year,race,gender,job_category,count,c_race,c_job_category
0,0,23andMe,2016,Latino,male,Executive/Senior officials & Mgrs,0,Non-White,Senior
1,1,23andMe,2016,Latino,male,First/Mid officials & Mgrs,1,Non-White,Senior
2,2,23andMe,2016,Latino,male,Professionals,7,Non-White,Senior
3,12,23andMe,2016,Latino,female,Executive/Senior officials & Mgrs,0,Non-White,Senior
4,13,23andMe,2016,Latino,female,First/Mid officials & Mgrs,1,Non-White,Senior
5,14,23andMe,2016,Latino,female,Professionals,5,Non-White,Senior
6,24,23andMe,2016,White,male,Executive/Senior officials & Mgrs,9,White,Senior
7,25,23andMe,2016,White,male,First/Mid officials & Mgrs,30,White,Senior
8,26,23andMe,2016,White,male,Professionals,61,White,Senior
9,36,23andMe,2016,Black_or_African_American,male,Executive/Senior officials & Mgrs,0,Non-White,Senior


At this point, we group our dataframe by gender and company.

In [10]:
d1=d1.groupby(['gender','company']).agg({'count':sum}).reset_index() #group d1 by gender and company
d1=d1.groupby(['company','gender']).agg({'count':sum}) 
d1=d1.unstack() #unstack d1
d1.head(10)

count       
gender   female   male
company               
23andMe     108    143
Adobe      1761   3808
Airbnb      604    923
Apple      8177  23335
Cisco      8274  24303
Facebook   3020   7202
Google    11622  31234
HP Inc.    3059   6395
HPE       13133  27529
Intel     11414  31565

We will caclulate the required proportions by gender for dataframe d1.

In [11]:
d1=d1['count']
d1=d1.iloc[:,:].apply(lambda x: (x/x.sum())*100,axis=1) #apply lamda function to caclulate d1 proportion over total rows
d1.columns=['Senior_Female','Senior_Male']
d1.head(10)

,Senior_Female,Senior_Male
company,,
23andMe,43.027888,56.972112
Adobe,31.621476,68.378524
Airbnb,39.554682,60.445318
Apple,25.948845,74.051155
Cisco,25.398287,74.601713
Facebook,29.544121,70.455879
Google,27.118723,72.881277
HP Inc.,32.356674,67.643326
HPE,32.297969,67.702031


We now perform the same operations on a second dataframe (d2) for Non-Senior positions.

In [12]:
d2 = workplace[workplace['c_job_category'] == 'Non-Senior'].reset_index() #d2 dataframe contains records with Non-Senior positions.
d2.head(10)

,index,company,year,race,gender,job_category,count,c_race,c_job_category
0,3,23andMe,2016,Latino,male,Technicians,0,Non-White,Non-Senior
1,4,23andMe,2016,Latino,male,Sales workers,0,Non-White,Non-Senior
2,5,23andMe,2016,Latino,male,Administrative support,0,Non-White,Non-Senior
3,6,23andMe,2016,Latino,male,Craft workers,0,Non-White,Non-Senior
4,7,23andMe,2016,Latino,male,operatives,0,Non-White,Non-Senior
5,8,23andMe,2016,Latino,male,laborers and helpers,0,Non-White,Non-Senior
6,9,23andMe,2016,Latino,male,Service workers,0,Non-White,Non-Senior
7,10,23andMe,2016,Latino,male,Totals,8,Non-White,Non-Senior
8,11,23andMe,2016,Latino,male,Previous_totals,0,Non-White,Non-Senior
9,15,23andMe,2016,Latino,female,Technicians,0,Non-White,Non-Senior



At this point, we group our dataframe by gender and company.

In [13]:
d2=d2.groupby(['gender','company']).agg({'count':sum}).reset_index()#group d2 by gender and company
d2=d2.groupby(['company','gender']).agg({'count':sum}) 
d2=d2.unstack()
d2

count        
gender     female    male
company                  
23andMe       190     153
Adobe        4900   10436
Airbnb       1688    2020
Apple       61046  134320
Cisco       20841   57948
Facebook     6639   14067
Google      26809   62526
HP Inc.     29182   60741
HPE         21257   42059
Intel       29845   89179
Intuit       6051    7807
LinkedIn     5895    8385
Lyft         1075    1369
MobileIron    337     824
Nvidia       1732    8536
Pinterest     724     929
Salesforce   9727   20716
Square        816    1397
Twitter      2404    4339
Uber         2098    4574
View          175     895
eBay         7811   11225

Now we have to caclulate the required proportions by gender for dataframe d1.

In [14]:
d2=d2['count']
d2=d2.iloc[:,:].apply(lambda x: (x/x.sum())*100,axis=1) #apply lamda function to caclulate d1 proportion over total rows
d2.columns=['Non_Senior_Female','Non_Senior_Male']
d2

,Non_Senior_Female,Non_Senior_Male
company,,
23andMe,55.393586,44.606414
Adobe,31.950965,68.049035
Airbnb,45.523193,54.476807
Apple,31.246993,68.753007
Cisco,26.451662,73.548338
Facebook,32.063170,67.936830
Google,30.009515,69.990485
HP Inc.,32.452209,67.547791
HPE,33.572873,66.427127


Now we join the two dataframes on company column and keep only the columns for males.

In [15]:
gender_df = d1.join(d2) #join d1 and d2 dataframes based on index
gender_df = gender_df.drop(['Senior_Female', 'Non_Senior_Female'], axis=1) #drop females proportions since they are of non-interest at this point
gender_df #now gender_df contains only senior and non-senior male proportions

,Senior_Male,Non_Senior_Male
company,,
23andMe,56.972112,44.606414
Adobe,68.378524,68.049035
Airbnb,60.445318,54.476807
Apple,74.051155,68.753007
Cisco,74.601713,73.548338
Facebook,70.455879,67.936830
Google,72.881277,69.990485
HP Inc.,67.643326,67.547791
HPE,67.702031,66.427127


We apply Analysis of Variance to reject / not reject the Null Hypothesis using Confidence Factor 95%.

H0: Proportions of males in senior positions is equal to proportions of males in non-senior positions.

H1: Proportions of males in senior positions is greater than proportions of males in non-senior positions.

We run our Two-Way ANOVA.

In [16]:
gen = smf.ols('Senior_Male ~ Non_Senior_Male', data=gender_df).fit() #fit anova model
aov_table = sm.stats.anova_lm(gen, typ=2) #run 2-way anova
print (aov_table) #print anova table

                     sum_sq    df          F        PR(>F)
Non_Senior_Male  818.331064   1.0  140.86266  1.651249e-10
Residual         116.188501  20.0        NaN           NaN


We got p-value=1.651249e-10 < a (let a = 0.05), so we reject the Null Hypothesis.

Consequently, the proportions of males in senior positions is greater than proportions of males in non-senior positions.

We will perform the same process for the race category.

In [17]:
d3 = workplace[workplace['c_job_category'] == 'Senior'].reset_index() #d3 dataframe will contain Senior position rows
d3.head(10) 

,index,company,year,race,gender,job_category,count,c_race,c_job_category
0,0,23andMe,2016,Latino,male,Executive/Senior officials & Mgrs,0,Non-White,Senior
1,1,23andMe,2016,Latino,male,First/Mid officials & Mgrs,1,Non-White,Senior
2,2,23andMe,2016,Latino,male,Professionals,7,Non-White,Senior
3,12,23andMe,2016,Latino,female,Executive/Senior officials & Mgrs,0,Non-White,Senior
4,13,23andMe,2016,Latino,female,First/Mid officials & Mgrs,1,Non-White,Senior
5,14,23andMe,2016,Latino,female,Professionals,5,Non-White,Senior
6,24,23andMe,2016,White,male,Executive/Senior officials & Mgrs,9,White,Senior
7,25,23andMe,2016,White,male,First/Mid officials & Mgrs,30,White,Senior
8,26,23andMe,2016,White,male,Professionals,61,White,Senior
9,36,23andMe,2016,Black_or_African_American,male,Executive/Senior officials & Mgrs,0,Non-White,Senior


At this point, we group dataframe d3 by gender and company.

In [18]:
d3=d3.groupby(['c_race','company']).agg({'count':sum}).reset_index() #group by company and c_race values
d3=d3.groupby(['company','c_race']).agg({'count':sum})
d3=d3.unstack() #unstack d3
d3.head(10) 

count       
c_race   Non-White  White
company                  
23andMe        340    162
Adobe         7608   3530
Airbnb        2228    826
Apple        45945  17079
Cisco        48754  16400
Facebook     15298   5146
Google       61694  24018
HP Inc.      11939   6969
HPE          51820  29504
Intel        65867  20091

We will calculate the proportions by gender for dataframe d3.

In [19]:
d3=d3['count']
d3=d3.iloc[:,:].apply(lambda x: (x/x.sum())*100,axis=1) #apply lamda function to calculate proportion for d3 dataframe.
d3.columns=['Senior_White','Senior_Non_White']
d3.head(10)

,Senior_White,Senior_Non_White
company,,
23andMe,67.729084,32.270916
Adobe,68.306698,31.693302
Airbnb,72.953504,27.046496
Apple,72.900800,27.099200
Cisco,74.828867,25.171133
Facebook,74.828801,25.171199
Google,71.978253,28.021747
HP Inc.,63.142585,36.857415
HPE,63.720427,36.279573


We will create d4 dataframe that contains records with Non-Senior positions.

In [20]:
d4 = workplace[workplace['c_job_category'] == 'Non-Senior'].reset_index() #d4 keeps Non-Senior records
d4.head(10)

,index,company,year,race,gender,job_category,count,c_race,c_job_category
0,3,23andMe,2016,Latino,male,Technicians,0,Non-White,Non-Senior
1,4,23andMe,2016,Latino,male,Sales workers,0,Non-White,Non-Senior
2,5,23andMe,2016,Latino,male,Administrative support,0,Non-White,Non-Senior
3,6,23andMe,2016,Latino,male,Craft workers,0,Non-White,Non-Senior
4,7,23andMe,2016,Latino,male,operatives,0,Non-White,Non-Senior
5,8,23andMe,2016,Latino,male,laborers and helpers,0,Non-White,Non-Senior
6,9,23andMe,2016,Latino,male,Service workers,0,Non-White,Non-Senior
7,10,23andMe,2016,Latino,male,Totals,8,Non-White,Non-Senior
8,11,23andMe,2016,Latino,male,Previous_totals,0,Non-White,Non-Senior
9,15,23andMe,2016,Latino,female,Technicians,0,Non-White,Non-Senior


At this point, we group dataframe d4 by gender and company.

In [21]:
d4=d4.groupby(['c_race','company']).agg({'count':sum}).reset_index() #group d4 dataframe by company and c_race
d4=d4.groupby(['company','c_race']).agg({'count':sum})
d4=d4.unstack() 
d4

count        
c_race     Non-White   White
company                     
23andMe          476     210
Adobe          20171   10501
Airbnb          5189    2227
Apple         278053  112679
Cisco         113437   44141
Facebook       30329   11083
Google        126616   52054
HP Inc.       115545   64301
HPE            80594   46038
Intel         175274   62774
Intuit         19822    7894
LinkedIn       20506    8054
Lyft            3319    1569
MobileIron      1661     661
Nvidia         16129    4407
Pinterest       2447     859
Salesforce     40007   20879
Square          3114    1312
Twitter         9428    4058
Uber            9813    3531
View            1514     626
eBay           26715   11357

We will calculate the proportions by gender for dataframe d4.

In [22]:
d4=d4['count']
d4=d4.iloc[:,:].apply(lambda x: (x/x.sum())*100,axis=1) #apply lamda function to calculate proportion for d4 dataframe
d4.columns=['Non_Senior_White','Non_Senior_Non_White']
d4

,Non_Senior_White,Non_Senior_Non_White
company,,
23andMe,69.387755,30.612245
Adobe,65.763563,34.236437
Airbnb,69.970334,30.029666
Apple,71.162075,28.837925
Cisco,71.987841,28.012159
Facebook,73.237226,26.762774
Google,70.865842,29.134158
HP Inc.,64.246633,35.753367
HPE,63.644261,36.355739


Now we join the two dataframes on company column (index) and keep only the columns for white people.

In [23]:
race_df = d3.join(d4) #join on index column (company)
race_df = race_df.drop(['Senior_Non_White', 'Non_Senior_Non_White'], axis=1) #drop non-white records which are of non-interest at this point
race_df

,Senior_White,Non_Senior_White
company,,
23andMe,67.729084,69.387755
Adobe,68.306698,65.763563
Airbnb,72.953504,69.970334
Apple,72.900800,71.162075
Cisco,74.828867,71.987841
Facebook,74.828801,73.237226
Google,71.978253,70.865842
HP Inc.,63.142585,64.246633
HPE,63.720427,63.644261


We apply Analysis of Variance to reject / not reject the Null Hypothesis using Confidence Factor 95%.

H0: Proportions of whites in senior positions is equal to proportions of whites in non-senior positions.

H1: Proportions of whites in senior positions is greater than proportions of whites in non-senior positions.

We run our Two-Way ANOVA.

In [24]:
race = smf.ols('Senior_White ~ Non_Senior_White', data=race_df).fit() #fit model
aov_table = sm.stats.anova_lm(race, typ=2) #run 2-way anova
print (aov_table)

                      sum_sq    df          F        PR(>F)
Non_Senior_White  280.035335   1.0  56.379785  3.059692e-07
Residual           99.338916  20.0        NaN           NaN


We got p-value=3.059692e-07 < a (let a = 0.05), so we reject the Null Hypothesis.

Consequently, the Proportions of whites in senior positions is greater than proportions of whites in non-senior positions.